In [456]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
# Import metrics module
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

In [457]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [458]:
df_train.head()

,VehicleID,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,"2,011",Silver,2.2,Nigerian Used,NaN
1,VHL18827,Ibadan,Hyundai,Sonata,"2,012",Silver,3.5,Nigerian Used,"125,000"
2,VHL19499,Lagos,Lexus,RX 350,"2,010",Red,9.2,Foreign Used,"110,852"
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,"2,017",Blue,22.8,Foreign Used,"30,000"
4,VHL12170,Ibadan,Toyota,Highlander,"2,002",Red,2.6,Nigerian Used,"125,206"


In [459]:
df_test.head()

,VehicleID,Location,Maker,Model,Year,Colour,Type,Distance
0,VHL18518,Abuja,BMW,323i,"2,008",White,Foreign Used,30524.0
1,VHL17149,Lagos,Toyota,Camry,"2,013",White,Foreign Used,NaN
2,VHL10927,Lagos,Toyota,Highlander Limited V6,"2,005",Gold,Foreign Used,NaN
3,VHL12909,Lagos,Toyota,Camry,"2,011",Gray,Foreign Used,166839.0
4,VHL12348,Lagos,Lexus,ES 350 FWD,"2,013",Red,Foreign Used,88862.0


In [460]:
# check column names for whitespaces
df_train.columns

Index(['VehicleID', 'Location', 'Maker', 'Model', 'Year', 'Colour',
       'Amount (Million Naira)', 'Type', 'Distance'],
      dtype='object')

In [461]:
# remove whitespace from Amount column name
df_train = df_train.rename(columns={'Amount (Million Naira)':'Amount'})
df_train.head()

,VehicleID,Location,Maker,Model,Year,Colour,Amount,Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,"2,011",Silver,2.2,Nigerian Used,NaN
1,VHL18827,Ibadan,Hyundai,Sonata,"2,012",Silver,3.5,Nigerian Used,"125,000"
2,VHL19499,Lagos,Lexus,RX 350,"2,010",Red,9.2,Foreign Used,"110,852"
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,"2,017",Blue,22.8,Foreign Used,"30,000"
4,VHL12170,Ibadan,Toyota,Highlander,"2,002",Red,2.6,Nigerian Used,"125,206"


In [462]:
# confirm colunm name change
df_train.columns

Index(['VehicleID', 'Location', 'Maker', 'Model', 'Year', 'Colour', 'Amount',
       'Type', 'Distance'],
      dtype='object')

In [463]:
# check data types of features of train dataset
df_train.dtypes

VehicleID     object
Location      object
Maker         object
Model         object
Year          object
Colour        object
Amount       float64
Type          object
Distance      object
dtype: object

In [464]:
df_train.replace(',','', regex=True, inplace=True)

In [465]:
df_train.head()

,VehicleID,Location,Maker,Model,Year,Colour,Amount,Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,2011,Silver,2.2,Nigerian Used,NaN
1,VHL18827,Ibadan,Hyundai,Sonata,2012,Silver,3.5,Nigerian Used,125000
2,VHL19499,Lagos,Lexus,RX 350,2010,Red,9.2,Foreign Used,110852
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,2017,Blue,22.8,Foreign Used,30000
4,VHL12170,Ibadan,Toyota,Highlander,2002,Red,2.6,Nigerian Used,125206


In [466]:
df_train.isnull().sum()

VehicleID       0
Location        0
Maker           0
Model           0
Year           21
Colour          0
Amount         17
Type          197
Distance     2360
dtype: int64

In [467]:
df_train['Year'] = df_train['Year'].fillna(2008)
df_train.isnull().sum()

VehicleID       0
Location        0
Maker           0
Model           0
Year            0
Colour          0
Amount         17
Type          197
Distance     2360
dtype: int64

In [468]:
df_train['Year'] = pd.to_datetime(df_train['Year'])
df_train['Year'] = pd.DatetimeIndex(df_train['Year']).year

In [469]:
# def option(data, col_name, choice):
    
#     output = data
#     missing = output[output[col_name].isnull()]
#     median = output.groupby(['Model','Year'])[col_name].median()
#     mean = output.groupby(['Model','Year'])[col_name].mean()
    

#     def fill_missing_median(df):
#         if pd.isnull(df[col_name]):
#             return median[df['Model'],df['Year']]
#         else:
#             return df[col_name]
        
#     def fill_missing_mean(df):
#         if pd.isnull(df[col_name]):
#             return mean[df['Model'],df['Year']]
#         else:
#             return df[col_name]
        
#     if choice == 'median':
#         output[col_name] = output.apply(fill_missing_median, axis=1)
#     elif choice == 'mean':
#         output[col_name] = output.apply(fill_missing_mean, axis=1)
        
#     return output

In [470]:
def option(data, choice='median'):
    
    output = data
    missing = output[output['Amount'].isnull()]
    median = output.groupby(['Model','Year'])['Amount'].median()

    def fill_missing_median(df):
        if pd.isnull(df['Amount']):
            return median[df['Model'],df['Year']]
        else:
            return df['Amount']
        
    if choice == 'median':
        output['Amount'] = output.apply(fill_missing_median, axis=1)

    return output

In [471]:
option(df_train, choice='median')

,VehicleID,Location,Maker,Model,Year,Colour,Amount,Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,2011,Silver,2.20,Nigerian Used,NaN
1,VHL18827,Ibadan,Hyundai,Sonata,2012,Silver,3.50,Nigerian Used,125000
2,VHL19499,Lagos,Lexus,RX 350,2010,Red,9.20,Foreign Used,110852
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,2017,Blue,22.80,Foreign Used,30000
4,VHL12170,Ibadan,Toyota,Highlander,2002,Red,2.60,Nigerian Used,125206
...,...,...,...,...,...,...,...,...,...
7200,VHL14329,Abuja,Honda,Civic,2018,Gray,5.70,Foreign Used,65000
7201,VHL10637,Abuja,BMW,X3,2007,White,4.00,Nigerian Used,200000
7202,VHL19734,Abuja,Toyota,RAV4 2.5 Limited 4x4,2010,Black,2.85,Nigerian Used,NaN
7203,VHL15569,Lagos,Mercedes-Benz,GLK-Class 350,2012,Black,8.65,Foreign Used,85750


In [472]:
df_train.isnull().sum()

VehicleID       0
Location        0
Maker           0
Model           0
Year            0
Colour          0
Amount          5
Type          197
Distance     2360
dtype: int64

In [473]:
df_train['Type'].value_counts()

Foreign Used     4896
Nigerian Used    1895
Brand New         217
Name: Type, dtype: int64

In [474]:
df_train['Type'] = df_train['Type'].fillna('Foreign Used')
df_train.isnull().sum()

VehicleID       0
Location        0
Maker           0
Model           0
Year            0
Colour          0
Amount          5
Type            0
Distance     2360
dtype: int64

In [475]:
df_train['Distance'] = df_train['Distance'].astype(float)

In [476]:
df_train.isnull().sum()

VehicleID       0
Location        0
Maker           0
Model           0
Year            0
Colour          0
Amount          5
Type            0
Distance     2360
dtype: int64

In [489]:
def option_mean(data, choice='mean'):
    
    output = data
    missing = output[output['Distance'].isnull()]
    mean = output.groupby(['Year','Type'])['Distance'].mean()

    def fill_missing_mean(df):
        if pd.isnull(df['Distance']):
            return mean[df['Year'],df['Type']]
        else:
            return df['Distance']
        
    if choice == 'mean':
        output['Distance'] = output.apply(fill_missing_mean, axis=1)

    return output

In [490]:
option_mean(df_train, choice='mean')

,VehicleID,Location,Maker,Model,Year,Colour,Amount,Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,2011,Silver,2.20,Nigerian Used,120351.776119
1,VHL18827,Ibadan,Hyundai,Sonata,2012,Silver,3.50,Nigerian Used,125000.000000
2,VHL19499,Lagos,Lexus,RX 350,2010,Red,9.20,Foreign Used,110852.000000
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,2017,Blue,22.80,Foreign Used,30000.000000
4,VHL12170,Ibadan,Toyota,Highlander,2002,Red,2.60,Nigerian Used,125206.000000
...,...,...,...,...,...,...,...,...,...
7200,VHL14329,Abuja,Honda,Civic,2018,Gray,5.70,Foreign Used,65000.000000
7201,VHL10637,Abuja,BMW,X3,2007,White,4.00,Nigerian Used,200000.000000
7202,VHL19734,Abuja,Toyota,RAV4 2.5 Limited 4x4,2010,Black,2.85,Nigerian Used,123295.340206
7203,VHL15569,Lagos,Mercedes-Benz,GLK-Class 350,2012,Black,8.65,Foreign Used,85750.000000


In [491]:
df_train.isnull().sum()

VehicleID    0
Location     0
Maker        0
Model        0
Year         0
Colour       0
Amount       0
Type         0
Distance     0
dtype: int64

In [492]:
mean_amount = df_train['Amount'].mean()
df_train['Amount'] = df_train['Amount'].fillna(mean_amount)

mean_dist = df_train['Distance'].mean()
df_train['Distance'] = df_train['Distance'].fillna(mean_dist)

In [493]:
df_train.isnull().sum()

VehicleID    0
Location     0
Maker        0
Model        0
Year         0
Colour       0
Amount       0
Type         0
Distance     0
dtype: int64

In [494]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   VehicleID  7205 non-null   object 
 1   Location   7205 non-null   object 
 2   Maker      7205 non-null   object 
 3   Model      7205 non-null   object 
 4   Year       7205 non-null   int64  
 5   Colour     7205 non-null   object 
 6   Amount     7205 non-null   float64
 7   Type       7205 non-null   object 
 8   Distance   7205 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 506.7+ KB


In [495]:
corr = df_train.corr()

In [496]:
corr['Amount'].sort_values(ascending=False)

Amount      1.000000
Year        0.415555
Distance   -0.219403
Name: Amount, dtype: float64

In [497]:
cat_df = [df_train['Type'], df_train['Amount']]
title_header = ['Type', 'Amount']
cat_df = pd.concat(cat_df, axis=1, keys=title_header)

In [498]:
cat_df['Type'] = cat_df['Type'].astype('category').cat.codes
cat_df_corr = cat_df.corr()
cat_df_corr['Amount'].sort_values(ascending=False)

Amount    1.000000
Type     -0.335943
Name: Amount, dtype: float64

In [499]:
ldt = [df_train["Location"], df_train["Amount"]]
headers = ["Location", "Amount"]
ldf = pd.concat(ldt, axis=1, keys=headers)

ldf['Location'].value_counts()

Lagos     3318
Abuja     3263
Ibadan     624
Name: Location, dtype: int64

In [500]:
ldf['Location'].head()

0     Abuja
1    Ibadan
2     Lagos
3     Abuja
4    Ibadan
Name: Location, dtype: object

In [501]:
ldf['Location'] = ldf['Location'].astype('category').cat.codes

In [502]:
ldf.head()

,Location,Amount
0,0,2.2
1,1,3.5
2,2,9.2
3,0,22.8
4,1,2.6


In [503]:
lcorr = ldf.corr()

lcorr['Amount'].sort_values(ascending=False)

Amount      1.000000
Location   -0.078426
Name: Amount, dtype: float64

In [504]:
mk_data = [df_train['Maker'], df_train['Amount']]
mk_header = ['Maker', 'Amount']
mk_data = pd.concat(mk_data, axis=1, keys=mk_header)

mk_data['Maker'].value_counts()

Toyota           2742
Lexus            1599
Mercedes-Benz    1213
Honda             358
Hyundai           162
Acura             154
Land Rover        124
Ford              120
BMW               111
Nissan            101
Peugeot            75
Kia                66
Volkswagen         54
Pontiac            30
Mazda              28
Dodge              24
Audi               23
Mitsubishi         23
Chevrolet          21
Infiniti           18
Jeep               18
Jaguar             12
Rolls-Royce        10
Mini                9
GMC                 8
Suzuki              8
Cadillac            7
Scion               7
Porsche             7
Volvo               7
Bentley             6
Maserati            6
Lincoln             6
Buick               5
Chrysler            4
Lamborghini         4
Opel                4
Rover               3
GAC                 3
Renault             3
Fiat                3
Citroen             2
Subaru              2
Saturn              2
JAC                 2
Hummer    

In [505]:
mk_data['Maker'] = mk_data['Maker'].astype('category').cat.codes
mcorr = mk_data.corr()

mcorr['Amount'].sort_values(ascending=False)

Amount    1.000000
Maker    -0.047972
Name: Amount, dtype: float64

In [506]:
c_data = [df_train["Colour"], df_train['Amount']]
c_headers = ["Colour", "Amount"]
c_data = pd.concat(c_data, axis=1, keys=c_headers)

c_data['Colour'] = c_data['Colour'].astype('category').cat.codes 
c_cor = c_data.corr()
# evaluating the correlation btw the features and the target variable
c_cor['Amount'].sort_values(ascending=False)

Amount    1.000000
Colour   -0.040738
Name: Amount, dtype: float64

In [507]:
mo_data = [df_train["Model"], df_train["Amount"]]
mo_header = ["Model", "Amount"]
mo_data = pd.concat(mo_data, axis=1, keys=mo_header)

mo_data['Model'] = mo_data['Model'].astype('category').cat.codes 
mo_corr = mo_data.corr()
# evaluating the correlation btw the features and the target variable
mo_corr['Amount'].sort_values(ascending=False)

Amount    1.000000
Model     0.085034
Name: Amount, dtype: float64

In [508]:
# New dataframe with all predictor variables
df_new = pd.concat([ldf['Location'], mk_data['Maker'], mo_data['Model'], 
                    df_train['Year'], c_data['Colour'], df_train['Type'], 
                    df_train['Distance']], axis=1)
df_new.head()

,Location,Maker,Model,Year,Colour,Type,Distance
0,0,17,117,2011,16,Nigerian Used,120351.776119
1,1,19,1049,2012,16,Nigerian Used,125000.000000
2,2,29,908,2010,15,Foreign Used,110852.000000
3,0,34,508,2017,2,Foreign Used,30000.000000
4,1,52,569,2002,15,Nigerian Used,125206.000000


In [509]:
df_new.isnull().sum()

Location    0
Maker       0
Model       0
Year        0
Colour      0
Type        0
Distance    0
dtype: int64

In [510]:
target = df_train['Amount']

In [511]:
target.head()

0     2.2
1     3.5
2     9.2
3    22.8
4     2.6
Name: Amount, dtype: float64

In [512]:
df_train = pd.get_dummies(df_new)

In [513]:
df_train.head()

,Location,Maker,Model,Year,Colour,Distance,Type_Brand New,Type_Foreign Used,Type_Nigerian Used
0,0,17,117,2011,16,120351.776119,0,0,1
1,1,19,1049,2012,16,125000.000000,0,0,1
2,2,29,908,2010,15,110852.000000,0,1,0
3,0,34,508,2017,2,30000.000000,0,1,0
4,1,52,569,2002,15,125206.000000,0,0,1


In [514]:
# Get maximum and minimum year of production of the cars
print(df_train['Year'].min(), df_train['Year'].max()) 

1970 2022


In [515]:
df_train['Year'].value_counts()

2008    755
2010    618
2013    599
2007    484
2012    460
2011    455
2009    439
2014    412
2015    369
2006    349
2005    298
2017    291
2016    280
2018    269
2004    222
2020    204
2019    189
2021    145
2003    122
2002     74
2001     55
2000     39
1970     21
1999     19
1998     11
2022     10
1997      5
1992      3
1986      2
1995      2
1996      1
1993      1
1979      1
1989      1
Name: Year, dtype: int64

In [516]:
df_train.loc[(df_train['Year'] >= 1970) & (df_train['Year'] <= 1975), 'Year'] = 0
df_train.loc[(df_train['Year'] > 1975) & (df_train['Year'] <= 1980), 'Year'] = 1
df_train.loc[(df_train['Year'] > 1980) & (df_train['Year'] <= 1985), 'Year'] = 2
df_train.loc[(df_train['Year'] > 1985) & (df_train['Year'] <= 1990), 'Year'] = 3
df_train.loc[(df_train['Year'] > 1990) & (df_train['Year'] <= 1995), 'Year'] = 4
df_train.loc[(df_train['Year'] > 1995) & (df_train['Year'] <= 2000), 'Year'] = 5
df_train.loc[(df_train['Year'] > 2000) & (df_train['Year'] <= 2005), 'Year'] = 6
df_train.loc[(df_train['Year'] > 2005) & (df_train['Year'] <= 2010), 'Year'] = 7
df_train.loc[(df_train['Year'] > 2010) & (df_train['Year'] <= 2015), 'Year'] = 8
df_train.loc[(df_train['Year'] > 2015) & (df_train['Year'] <= 2020), 'Year'] = 9
df_train.loc[ df_train['Year'] > 2020, 'Year'] = 10

In [517]:
df_train.head()

,Location,Maker,Model,Year,Colour,Distance,Type_Brand New,Type_Foreign Used,Type_Nigerian Used
0,0,17,117,8,16,120351.776119,0,0,1
1,1,19,1049,8,16,125000.000000,0,0,1
2,2,29,908,7,15,110852.000000,0,1,0
3,0,34,508,9,2,30000.000000,0,1,0
4,1,52,569,6,15,125206.000000,0,0,1


In [518]:
for column in df_train.columns:
    df_train[column] = (df_train[column] - df_train[column].min()) / (df_train[column].max() - df_train[column].min())

In [519]:
df_train.head()

,Location,Maker,Model,Year,Colour,Distance,Type_Brand New,Type_Foreign Used,Type_Nigerian Used
0,0.0,0.314815,0.095745,0.8,0.80,0.060618,0.0,0.0,1.0
1,0.5,0.351852,0.858429,0.8,0.80,0.062959,0.0,0.0,1.0
2,1.0,0.537037,0.743044,0.7,0.75,0.055833,0.0,1.0,0.0
3,0.0,0.629630,0.415712,0.9,0.10,0.015110,0.0,1.0,0.0
4,0.5,0.962963,0.465630,0.6,0.75,0.063063,0.0,0.0,1.0


In [524]:
df_test.head(50)

,VehicleID,Location,Maker,Model,Year,Colour,Type,Distance
0,VHL18518,Abuja,BMW,323i,"2,008",White,Foreign Used,30524.0
1,VHL17149,Lagos,Toyota,Camry,"2,013",White,Foreign Used,NaN
2,VHL10927,Lagos,Toyota,Highlander Limited V6,"2,005",Gold,Foreign Used,NaN
3,VHL12909,Lagos,Toyota,Camry,"2,011",Gray,Foreign Used,166839.0
4,VHL12348,Lagos,Lexus,ES 350 FWD,"2,013",Red,Foreign Used,88862.0
5,VHL10798,Lagos,Mercedes-Benz,GLE-Class,"2,017",Black,Foreign Used,69843.0
6,VHL11022,Abuja,Toyota,Sienna LE 4dr,"1,999",Gold,Foreign Used,97000.0
7,VHL12206,Lagos,Lexus,ES 350,"2,011",Red,Foreign Used,NaN
8,VHL11697,Abuja,Honda,CR-V,"2,017",White,Foreign Used,80677.0
9,VHL12313,Lagos,Toyota,Camry,"2,013",Black,Foreign Used,NaN


In [525]:
df_test.isnull().sum()

VehicleID      0
Location       0
Maker          0
Model          0
Year           2
Colour         0
Type          54
Distance     676
dtype: int64

In [526]:
df_test.replace(',','', regex=True, inplace=True)

In [527]:
df_test['Year'].mode()

0    2008
dtype: object

In [528]:
df_test['Year'] = df_test['Year'].fillna(2008)

In [529]:
df_test['Type'].value_counts()

Foreign Used     1390
Nigerian Used     547
Brand New          70
Name: Type, dtype: int64

In [530]:
df_test['Type'] = df_test['Type'].fillna('Foreign Used')

In [531]:
df_test['Type'].value_counts()

Foreign Used     1444
Nigerian Used     547
Brand New          70
Name: Type, dtype: int64

In [532]:
df_test.isnull().sum()

VehicleID      0
Location       0
Maker          0
Model          0
Year           0
Colour         0
Type           0
Distance     676
dtype: int64

In [533]:
df_test['Distance'] = df_test['Distance'].astype(float)

In [534]:
option_mean(df_test, choice='mean')

,VehicleID,Location,Maker,Model,Year,Colour,Type,Distance
0,VHL18518,Abuja,BMW,323i,2008,White,Foreign Used,30524.000000
1,VHL17149,Lagos,Toyota,Camry,2013,White,Foreign Used,73704.964286
2,VHL10927,Lagos,Toyota,Highlander Limited V6,2005,Gold,Foreign Used,122717.228571
3,VHL12909,Lagos,Toyota,Camry,2011,Gray,Foreign Used,166839.000000
4,VHL12348,Lagos,Lexus,ES 350 FWD,2013,Red,Foreign Used,88862.000000
...,...,...,...,...,...,...,...,...
2056,VHL17903,Abuja,Toyota,Avalon,2020,Red,Foreign Used,2650.000000
2057,VHL14018,Ibadan,Audi,Q5,2011,Blue,Foreign Used,99000.000000
2058,VHL17473,Ibadan,Toyota,Camry,2015,Black,Foreign Used,108000.000000
2059,VHL11480,Abuja,Lexus,IS 350 AWD,2013,White,Foreign Used,52485.000000


In [535]:
df_test.isnull().sum()

VehicleID    0
Location     0
Maker        0
Model        0
Year         0
Colour       0
Type         0
Distance     4
dtype: int64

In [536]:
mean_dist_test = df_test['Distance'].mean()
df_test['Distance'] = df_test['Distance'].fillna(mean_dist_test)

In [537]:
df_test.isnull().sum()

VehicleID    0
Location     0
Maker        0
Model        0
Year         0
Colour       0
Type         0
Distance     0
dtype: int64

In [538]:
test_id = df_test[['VehicleID']]

In [539]:
df_test = df_test.drop(['VehicleID'], axis=1)

In [540]:
df_test.head()

,Location,Maker,Model,Year,Colour,Type,Distance
0,Abuja,BMW,323i,2008,White,Foreign Used,30524.000000
1,Lagos,Toyota,Camry,2013,White,Foreign Used,73704.964286
2,Lagos,Toyota,Highlander Limited V6,2005,Gold,Foreign Used,122717.228571
3,Lagos,Toyota,Camry,2011,Gray,Foreign Used,166839.000000
4,Lagos,Lexus,ES 350 FWD,2013,Red,Foreign Used,88862.000000


In [541]:
df_test['Year'] = pd.to_datetime(df_test['Year'])
df_test['Year'] = pd.DatetimeIndex(df_test['Year']).year

In [542]:
df_test['Location'] = df_test['Location'].astype('category').cat.codes
df_test['Maker'] = df_test['Maker'].astype('category').cat.codes
df_test['Model'] = df_test['Model'].astype('category').cat.codes
df_test['Colour'] = df_test['Colour'].astype('category').cat.codes

df_test.head()

,Location,Maker,Model,Year,Colour,Type,Distance
0,0,2,8,2008,16,Foreign Used,30524.000000
1,2,37,123,2013,16,Foreign Used,73704.964286
2,2,37,272,2005,7,Foreign Used,122717.228571
3,2,37,123,2011,8,Foreign Used,166839.000000
4,2,20,192,2013,12,Foreign Used,88862.000000


In [543]:
df_test = pd.get_dummies(df_test)

In [544]:
df_test.head()

,Location,Maker,Model,Year,Colour,Distance,Type_Brand New,Type_Foreign Used,Type_Nigerian Used
0,0,2,8,2008,16,30524.000000,0,1,0
1,2,37,123,2013,16,73704.964286,0,1,0
2,2,37,272,2005,7,122717.228571,0,1,0
3,2,37,123,2011,8,166839.000000,0,1,0
4,2,20,192,2013,12,88862.000000,0,1,0


In [545]:
df_test.loc[(df_test['Year'] >= 1970) & (df_test['Year'] <= 1975), 'Year'] = 0
df_test.loc[(df_test['Year'] > 1975) & (df_test['Year'] <= 1980), 'Year'] = 1
df_test.loc[(df_test['Year'] > 1980) & (df_test['Year'] <= 1985), 'Year'] = 2
df_test.loc[(df_test['Year'] > 1985) & (df_test['Year'] <= 1990), 'Year'] = 3
df_test.loc[(df_test['Year'] > 1990) & (df_test['Year'] <= 1995), 'Year'] = 4
df_test.loc[(df_test['Year'] > 1995) & (df_test['Year'] <= 2000), 'Year'] = 5
df_test.loc[(df_test['Year'] > 2000) & (df_test['Year'] <= 2005), 'Year'] = 6
df_test.loc[(df_test['Year'] > 2005) & (df_test['Year'] <= 2010), 'Year'] = 7
df_test.loc[(df_test['Year'] > 2010) & (df_test['Year'] <= 2015), 'Year'] = 8
df_test.loc[(df_test['Year'] > 2015) & (df_test['Year'] <= 2020), 'Year'] = 9
df_test.loc[ df_test['Year'] > 2020, 'Year'] = 10

In [546]:
df_test.head()

,Location,Maker,Model,Year,Colour,Distance,Type_Brand New,Type_Foreign Used,Type_Nigerian Used
0,0,2,8,7,16,30524.000000,0,1,0
1,2,37,123,8,16,73704.964286,0,1,0
2,2,37,272,6,7,122717.228571,0,1,0
3,2,37,123,8,8,166839.000000,0,1,0
4,2,20,192,8,12,88862.000000,0,1,0


In [547]:
for column in df_test.columns:
    df_test[column] = (df_test[column] - df_test[column].min()) / (df_test[column].max() - 
                                                                   df_test[column].min())

In [548]:
df_test.head()

,Location,Maker,Model,Year,Colour,Distance,Type_Brand New,Type_Foreign Used,Type_Nigerian Used
0,0.0,0.051282,0.013652,0.7,0.941176,0.030981,0.0,1.0,0.0
1,1.0,0.948718,0.209898,0.8,0.941176,0.074810,0.0,1.0,0.0
2,1.0,0.948718,0.464164,0.6,0.411765,0.124558,0.0,1.0,0.0
3,1.0,0.948718,0.209898,0.8,0.470588,0.169342,0.0,1.0,0.0
4,1.0,0.512821,0.327645,0.8,0.705882,0.090195,0.0,1.0,0.0


### Modeling

In [549]:
X = df_train.copy()
y = target

In [550]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

In [551]:
lm = LinearRegression()
lm.fit(X_train, y_train)
lm_pred = lm.predict(X_test)

In [552]:
print('Linear Model: ', metrics.mean_squared_error(y_test, lm_pred))

Linear Model:  463.1200453147281


In [553]:
import xgboost as xgb
xg = xgb.XGBRegressor(n_estimators = 300)
xg.fit(X_train, y_train)
xgb_pred = xg.predict(X_test)

In [554]:
print('XGBoost Model: ', metrics.mean_squared_error(y_test, xgb_pred))

XGBoost Model:  243.39954242430463


In [555]:
df_test.shape

(2061, 9)

In [556]:
df_test.head()

,Location,Maker,Model,Year,Colour,Distance,Type_Brand New,Type_Foreign Used,Type_Nigerian Used
0,0.0,0.051282,0.013652,0.7,0.941176,0.030981,0.0,1.0,0.0
1,1.0,0.948718,0.209898,0.8,0.941176,0.074810,0.0,1.0,0.0
2,1.0,0.948718,0.464164,0.6,0.411765,0.124558,0.0,1.0,0.0
3,1.0,0.948718,0.209898,0.8,0.470588,0.169342,0.0,1.0,0.0
4,1.0,0.512821,0.327645,0.8,0.705882,0.090195,0.0,1.0,0.0


In [559]:
xg_pred_new = xg.predict(df_test)  ##PREDICTS FOR OUR TEST DATAFRAME
df_sub = df_test.copy() #CREATES A COPY OF OUR TEST DATAFRAME

df_sub['Amount(MillionNaira)'] = xg_pred_new #CREATES A NEW COLUMN 'Amount(MillionNaira)' AND SAVES OUR PREDICTED PRICES THERE
data = test_id #FOR THE ORIGINAL INDEXES WE DROPPED

data = pd.concat([data, df_sub[['Amount(MillionNaira)']]], axis=1) #JOINS THE INDEX DATAFRAME WITH THE PREDICTION DATA FRAME

data.to_csv('results.csv', header=True, index=False) #SAVES TO CSV